# 가볍게 2020to2023

In [3]:
import pandas as pd
import numpy as np
import torch

In [1]:
from konlpy.tag import Mecab

In [115]:
DF = pd.read_csv("melon_chart2020to2023.csv")
DF

,Unnamed: 0,가수,곡명,가사,장르,발매일
0,0,지코 (ZICO),아무노래,왜들 그리 다운돼있어\n뭐가 문제야 say something\n분위기가 겁나 싸해\...,랩/힙합,2020.01.13
1,1,창모 (CHANGMO),METEOR,Yeah ha\nYeah ha\nYeah ha\nYeah ha\nYeah ha\nY...,랩/힙합,2019.11.29
2,2,조정석,아로하,어두운 불빛아래 촛불 하나\n와인 잔에 담긴 약속하나\n항상 너의 곁에서 널 지켜줄...,"발라드, 국내드라마",2020.03.27
3,3,장범준,흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야,흔들리는 꽃들 속에서\n네 샴푸향이 느껴진거야\n스쳐지나간건가 뒤돌아보지만\n그냥 ...,"록/메탈, 국내드라마",2019.08.23
4,4,아이유,에잇(Prod.&Feat. SUGA of BTS),So are you happy now\nFinally happy now are yo...,록/메탈,2020.05.06
...,...,...,...,...,...,...
394,394,성시경,희재,햇살은 우릴 위해 내리고\n바람도 서롤 감싸게 했죠\n우리 웃음속에\n계절은 오고 ...,"발라드, 국내영화",2003.02.01
395,395,임한별,떠나보낼 준비해 둘걸 그랬어,너랑 입으려 고른 옷 오늘 받아봤어\n오래 걸려서 잊고 있었는데\n곧 있을 너의 생...,발라드,2022.10.18
396,396,싸이 (PSY),That That (prod. & feat. SUGA of BTS),20 22\nPSY Coming back (이리 오너라)\n\nLong time n...,댄스,2022.04.29
397,397,BE'O (비오),자격지심 (Feat. ZICO),내 나이는 20\n하고도 플러스 2지\n주위 친구들은 일찍\n만들어 본인의 PPT\...,랩/힙합,2022.09.29


In [7]:
lyrics = DF[['가사']]

# 2. 데이터 전처리

In [8]:
# 함수 파일이 변경돼도 다시 import 안해도 됨
%load_ext autoreload
%autoreload 2

from NLP_preprocess import drop_row, regex_processing, clean_by_freq, clean_by_stopwords, clean_by_len, idx_encoder

## 2-1 결측치, 중복 데이터 처리

In [10]:
lyrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   가사      399 non-null    object
dtypes: object(1)
memory usage: 3.2+ KB


## 2-2 텍스트 데이터 전처리 
- 정제(불용어, 노이즈 제거)
- 토큰화
- 정제
-> 단어사전

In [12]:
hangule_pattern = "[^ㄱ-ㅎㅏ-ㅣ가-힣 ]" # 한글 및 공백만 남기기
lyrics["가사"] = lyrics["가사"].apply(lambda x: regex_processing(x, hangule_pattern))

lyrics

C:\Users\kdp\AppData\Local\Temp\ipykernel_15764\3357944842.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyrics["가사"] = lyrics["가사"].apply(lambda x: regex_processing(x, hangule_pattern))


,가사
0,왜들 그리 다운돼있어뭐가 문제야 분위기가 겁나 싸해요새는 이런 게 유행인가왜들 그...
1,모두 그에게 말해 또 왔네새꺄 죽지 않고 왔다 이렇게새꺄 뭐 ...
2,어두운 불빛아래 촛불 하나와인 잔에 담긴 약속하나항상 너의 곁에서 널 지켜줄거야날 ...
3,흔들리는 꽃들 속에서네 샴푸향이 느껴진거야스쳐지나간건가 뒤돌아보지만그냥 사람들만 보...
4,뭐 그대로야 난다 잃어버린 것 같아모든 게 맘대로 왔다가 인사도 없이...
...,...
394,햇살은 우릴 위해 내리고바람도 서롤 감싸게 했죠우리 웃음속에계절은 오고 또 갔죠바람...
395,너랑 입으려 고른 옷 오늘 받아봤어오래 걸려서 잊고 있었는데곧 있을 너의 생일에 예...
396,이리 오너라 오래간만이지 우리 다시 웃고 울고 지지고 볶고 그래 ...
397,내 나이는 하고도 플러스 지주위 친구들은 일찍만들어 본인의 말도 안 돼 거짓말내게 ...


In [35]:
# 단어 토큰화
from konlpy.tag import Okt

okt = Okt()
print(okt.pos("한국어 형태소 분석을 시작합니다."))

lyrics["tokened_document"] = lyrics["가사"].apply(lambda x: okt.morphs(x))

[('한국어', 'Noun'), ('형태소', 'Noun'), ('분석', 'Noun'), ('을', 'Josa'), ('시작', 'Noun'), ('합니다', 'Verb'), ('.', 'Punctuation')]


C:\Users\kdp\AppData\Local\Temp\ipykernel_15764\3896951950.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyrics["tokened_document"] = lyrics["가사"].apply(lambda x: okt.morphs(x))


In [36]:
lyrics

,가사,tokened_document
0,왜들 그리 다운돼있어뭐가 문제야 분위기가 겁나 싸해요새는 이런 게 유행인가왜들 그...,"[왜, 들, 그리, 다운, 돼있어, 뭐, 가, 문제, 야, 분위기, 가, 겁나, 싸..."
1,모두 그에게 말해 또 왔네새꺄 죽지 않고 왔다 이렇게새꺄 뭐 ...,"[모두, 그, 에게, 말, 해, 또, 왔, 네, 새, 꺄, 죽지, 않고, 왔다, 이..."
2,어두운 불빛아래 촛불 하나와인 잔에 담긴 약속하나항상 너의 곁에서 널 지켜줄거야날 ...,"[어, 두운, 불빛, 아래, 촛불, 하나, 와인, 잔, 에, 담긴, 약속, 하나, ..."
3,흔들리는 꽃들 속에서네 샴푸향이 느껴진거야스쳐지나간건가 뒤돌아보지만그냥 사람들만 보...,"[흔들리는, 꽃, 들, 속, 에서, 네, 샴푸, 향, 이, 느껴진거야, 스쳐, 지나..."
4,뭐 그대로야 난다 잃어버린 것 같아모든 게 맘대로 왔다가 인사도 없이...,"[뭐, 그대로, 야, 난다, 잃어버린, 것, 같아, 모든, 게, 맘대로, 왔다가, ..."
...,...,...
394,햇살은 우릴 위해 내리고바람도 서롤 감싸게 했죠우리 웃음속에계절은 오고 또 갔죠바람...,"[햇살, 은, 우릴, 위해, 내리고, 바람, 도, 서, 롤, 감싸게, 했죠, 우리,..."
395,너랑 입으려 고른 옷 오늘 받아봤어오래 걸려서 잊고 있었는데곧 있을 너의 생일에 예...,"[너, 랑, 입으려, 고른, 옷, 오늘, 받아, 봤어오래, 걸려서, 잊고, 있었는데..."
396,이리 오너라 오래간만이지 우리 다시 웃고 울고 지지고 볶고 그래 ...,"[이리, 오, 너, 라, 오래간만, 이지, 우리, 다시, 웃고, 울, 고, 지지, ..."
397,내 나이는 하고도 플러스 지주위 친구들은 일찍만들어 본인의 말도 안 돼 거짓말내게 ...,"[내, 나이, 는, 하고도, 플러스, 지, 주위, 친구, 들, 은, 일찍, 만들어,..."


In [37]:
# from collections import Counter
# 
# tokens = lyrics['tokened_document'][4]
# 
# vocab = Counter(tokens)
# vocab = vocab.most_common()
# 
# print(vocab)

In [38]:
# 등장 빈도 기준 정제 함수
# -> 노래 가사라서 벌스 부분은 아무래도 중복이 적음
# -> 등장 빈도 기준 정제는 생략 

In [39]:
# 길이 기준 -> 1은 없애자 
lyrics["tokened_document"] = lyrics["tokened_document"].apply(lambda x: clean_by_len(x, 1))

lyrics["tokened_document"]

C:\Users\kdp\AppData\Local\Temp\ipykernel_15764\1196562747.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyrics["tokened_document"] = lyrics["tokened_document"].apply(lambda x: clean_by_len(x, 1))


0      [그리, 다운, 돼있어, 문제, 분위기, 겁나, 요새, 이런, 유행, 인가, 그리,...
1      [모두, 에게, 죽지, 않고, 왔다, 이렇게, 의라, 빌릴래, 했어, 예수, 처럼,...
2      [두운, 불빛, 아래, 촛불, 하나, 와인, 담긴, 약속, 하나, 항상, 에서, 지...
3      [흔들리는, 에서, 샴푸, 느껴진거야, 스쳐, 지나간, 건가, 돌아보지만, 그냥, ...
4      [그대로, 난다, 잃어버린, 같아, 모든, 맘대로, 왔다가, 인사, 없이, 떠나이대...
                             ...                        
394    [햇살, 우릴, 위해, 내리고, 바람, 감싸게, 했죠, 우리, 웃음, 계절, 오고,...
395    [입으려, 고른, 오늘, 받아, 봤어오래, 걸려서, 잊고, 있었는데, 있을, 생일,...
396    [이리, 오래간만, 이지, 우리, 다시, 웃고, 지지, 볶고, 그래, 기분, 오져,...
397    [나이, 하고도, 플러스, 주위, 친구, 일찍, 만들어, 본인, 거짓말, 내게, 하...
398    [이듬해, 피는, 한여름, 가을, 겨울, 내릴, 또다시, 봄정, 들었던, 젊은, 이...
Name: tokened_document, Length: 399, dtype: object

In [40]:
lyrics["tokened_document"] = lyrics["tokened_document"].apply(lambda x: clean_by_stopwords(x))

# clean_by_stopwords(tokens1) # 두번째 인자 안주면 그냥 default
# type(tokens1), len(tokens1)

C:\Users\kdp\AppData\Local\Temp\ipykernel_15764\2357069751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyrics["tokened_document"] = lyrics["tokened_document"].apply(lambda x: clean_by_stopwords(x))


In [41]:
lyrics["tokened_document"] # 정제 끝?!

0      [그리, 다운, 돼있어, 문제, 분위기, 겁나, 요새, 유행, 인가, 그리, 재미없...
1      [죽지, 않고, 왔다, 이렇게, 의라, 빌릴래, 했어, 예수, 처럼, 이렇게, 부활...
2      [두운, 불빛, 아래, 촛불, 와인, 담긴, 약속, 항상, 지켜줄거야, 믿어, 였잖...
3      [흔들리는, 샴푸, 느껴진거야, 스쳐, 지나간, 건가, 돌아보지만, 그냥, 사람, ...
4      [그대로, 난다, 잃어버린, 같아, 모든, 맘대로, 왔다가, 인사, 없이, 떠나이대...
                             ...                        
394    [햇살, 우릴, 위해, 내리고, 바람, 감싸게, 했죠, 웃음, 계절, 오고, 갔죠,...
395    [입으려, 고른, 오늘, 받아, 봤어오래, 걸려서, 잊고, 있었는데, 있을, 생일,...
396    [이리, 오래간만, 이지, 다시, 웃고, 지지, 볶고, 기분, 오져, 다시, 오죠,...
397    [나이, 하고도, 플러스, 주위, 친구, 일찍, 만들어, 본인, 거짓말, 내게, 하...
398    [이듬해, 피는, 한여름, 가을, 겨울, 내릴, 또다시, 봄정, 들었던, 젊은, 이...
Name: tokened_document, Length: 399, dtype: object

In [87]:
import json

with open('SentiWord_info.json', encoding='utf-8-sig', mode='r') as f: 
  SentiWord_info = json.load(f)

sentiword_DF = pd.DataFrame(SentiWord_info)
sentiword_DF

,word,word_root,polarity
0,(-;,(,1
1,(;_;),(;_;),-1
2,(^^),(^^),1
3,(^-^),(^-^),1
4,(^^*,(,1
...,...,...,...
14849,반신반의하다,반신반의,0
14850,신비롭다,신비,1
14851,아리송하다,아리송,-1
14852,알쏭하다,알쏭하,-1


In [90]:
sentiword_dic = {k:v for k,v in zip(sentiword_DF["word"], sentiword_DF["polarity"])}
sentiword_dic

{'(-;': '1',
 '(;_;)': '-1',
 '(^^)': '1',
 '(^-^)': '1',
 '(^^*': '1',
 '(^_^)': '1',
 '(^_^;': '-1',
 '(^o^)': '1',
 '(-_-)': '-1',
 '(T_T)': '-1',
 '(ㅡㅡ)': '-1',
 ')-:': '-1',
 '*^^*': '1',
 '/^o^\\': '1',
 ':(': '1',
 ":'-(": '1',
 ':-(': '1',
 ':)': '-1',
 ':-)': '1',
 ':/': '-1',
 ':|': '1',
 ':D': '-1',
 ':-D': '-1',
 ':p': '0',
 ':-P': '-1',
 ';': '1',
 ';)': '1',
 ';-)': '1',
 ';ㅅ;': '-1',
 '^///^': '1',
 '^^': '1',
 '^^//': '1',
 '^-^;': '-1',
 '^-^;;': '-1',
 '^-^;;;;;': '-1',
 '-_-^': '-1',
 '<3': '1',
 '=)': '1',
 '8-)': '0',
 'B-)': '0',
 'XD': '1',
 'ㄱㅅ': '1',
 'ㄱㅇㄷ': '1',
 '가격이 싸다': '1',
 '가까스로': '0',
 '가까이 사귀어': '1',
 '가까이하다': '1',
 '가꾸러뜨리다': '-1',
 '가꾸러트리다': '-1',
 '가난': '-2',
 '가난뱅이': '-2',
 '가난살이': '-2',
 '가난살이하다': '-2',
 '가난설음': '-2',
 '가난에': '-2',
 '가난에 쪼들려서': '-2',
 '가난하게': '-2',
 '가난하고': '-2',
 '가난하고 어렵다': '-2',
 '가난하고 천한': '-2',
 '가난하다': '-2',
 '가난하여': '-2',
 '가난한': '-2',
 '가난한 데다가': '-2',
 '가난한 사람을': '-2',
 '가난히': '-2',
 '가년스럽다': '-2',
 '가념하다': '-1',
 '가녕스럽다':

In [109]:
"고생을 격다" in sentiword_dic.keys()

True

In [112]:
# 딕셔너리 ver 
def sentiment_point(tokens, sentiword_dic):
  '''
  
  :param tokens: token list
  :param sentiword_dic: dataframe
  :return: 
  '''
  # print(sentiword_dic["word"].str.contains("(^^)"))

  # print(type(tokens)) # list 
  sentiment = 0
  for token in tokens:
    
    if token in sentiword_dic.keys():
      # print(sum(sentiword_dic["word"].isin([token])))
      # print(token)
      
        sentiment += int(sentiword_dic[token])
    else:
      continue 
  
  return sentiment
      
# lyrics["sentiment"]
lyrics["sentiment"] = lyrics["tokened_document"].iloc[:].apply(lambda x: sentiment_point(x, sentiword_dic))

C:\Users\kdp\AppData\Local\Temp\ipykernel_15764\3605357798.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyrics["sentiment"] = lyrics["tokened_document"].iloc[:].apply(lambda x: sentiment_point(x, sentiword_dic))


In [97]:
# my_lyrics = lyrics["tokened_document"][0]
# my_lyrics

In [114]:
lyrics

,가사,tokened_document,sentiment
0,왜들 그리 다운돼있어뭐가 문제야 분위기가 겁나 싸해요새는 이런 게 유행인가왜들 그...,"[그리, 다운, 돼있어, 문제, 분위기, 겁나, 요새, 유행, 인가, 그리, 재미없...",-2
1,모두 그에게 말해 또 왔네새꺄 죽지 않고 왔다 이렇게새꺄 뭐 ...,"[죽지, 않고, 왔다, 이렇게, 의라, 빌릴래, 했어, 예수, 처럼, 이렇게, 부활...",-15
2,어두운 불빛아래 촛불 하나와인 잔에 담긴 약속하나항상 너의 곁에서 널 지켜줄거야날 ...,"[두운, 불빛, 아래, 촛불, 와인, 담긴, 약속, 항상, 지켜줄거야, 믿어, 였잖...",4
3,흔들리는 꽃들 속에서네 샴푸향이 느껴진거야스쳐지나간건가 뒤돌아보지만그냥 사람들만 보...,"[흔들리는, 샴푸, 느껴진거야, 스쳐, 지나간, 건가, 돌아보지만, 그냥, 사람, ...",-1
4,뭐 그대로야 난다 잃어버린 것 같아모든 게 맘대로 왔다가 인사도 없이...,"[그대로, 난다, 잃어버린, 같아, 모든, 맘대로, 왔다가, 인사, 없이, 떠나이대...",-6
...,...,...,...
394,햇살은 우릴 위해 내리고바람도 서롤 감싸게 했죠우리 웃음속에계절은 오고 또 갔죠바람...,"[햇살, 우릴, 위해, 내리고, 바람, 감싸게, 했죠, 웃음, 계절, 오고, 갔죠,...",8
395,너랑 입으려 고른 옷 오늘 받아봤어오래 걸려서 잊고 있었는데곧 있을 너의 생일에 예...,"[입으려, 고른, 오늘, 받아, 봤어오래, 걸려서, 잊고, 있었는데, 있을, 생일,...",-6
396,이리 오너라 오래간만이지 우리 다시 웃고 울고 지지고 볶고 그래 ...,"[이리, 오래간만, 이지, 다시, 웃고, 지지, 볶고, 기분, 오져, 다시, 오죠,...",-4
397,내 나이는 하고도 플러스 지주위 친구들은 일찍만들어 본인의 말도 안 돼 거짓말내게 ...,"[나이, 하고도, 플러스, 주위, 친구, 일찍, 만들어, 본인, 거짓말, 내게, 하...",2
